In [ ]:
# Proyecto web scraping de Google Maps. Si quieres ver más ejemplos entra en la academia https://scraping.link/
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
import time
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')



class ScrapearGMaps:
    
    data = {}
    worksheet = {}
    
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome('chromedriver',options=options)
        
        now = datetime.now()
        today = now.strftime("%Y-%m-%d")

        gc = gspread.authorize(GoogleCredentials.get_application_default())

        # Abrir por titulo
        sh = gc.open("Veterinarios")

        # Seleccionar primera hoja
        self.worksheet = sh.get_worksheet(0)
    
    def scroll_the_page(self, i):
        try:
            section_loading = self.driver.find_element_by_class_name("section-loading")
            while True:
                if i >= len(self.driver.find_elements_by_class_name("place-result-container-place-link")):
                    actions = ActionChains(self.driver)
                    actions.move_to_element(section_loading).perform()
                    time.sleep(2)
                else:
                    break
        except:
            pass
        
    def get_geocoder(self, url_location): # gets geographical lat/long coordinates
        try:
            coords = re.search(r"!3d-?\d\d?\.\d{4,8}!4d-?\d\d?\.\d{4,8}",
                            url_location).group()
            coord = coords.split('!3d')[1]
            return tuple(coord.split('!4d'))
        except (TypeError, AttributeError):
            return ("", "")
        
    def get_name(self):
        try:
            return self.driver.find_element_by_xpath("//h1[contains(@class,'header-title')]").text
        except:
            return ""
        
    def get_address(self):
        try:
            return self.driver.find_element_by_css_selector("[data-item-id='address']").text
        except:
            return ""
        
    def get_phone(self):
        try:
            return self.driver.find_element_by_xpath("//button[contains(@data-item-id,'phone')]").text
        except:
            return ""
        
    def get_website(self):
        try:
            return self.driver.find_element_by_css_selector("[data-item-id='authority']").text
        except:
            return ""

    
    def scrape(self, url):
        try:
            self.driver.get(url)
            #print(self.driver.page_source)
            time.sleep(2)
            
            #element = self.driver.find_element_by_xpath("//button[.//span[text()='Acepto']]")
            #element.click()
            
            time.sleep(3)
                        
            for i in range(0,20):
                self.scroll_the_page(i)                
                
                place = self.driver.find_elements_by_class_name("place-result-container-place-link")[i]                
                place.click()
                
                time.sleep(3)
                
                name = self.get_name()
                address = self.get_address()
                phone_number = self.get_phone()
                website = self.get_website()
                coords = self.get_geocoder(self.driver.current_url)
                email = ""
                #if website != "":
                #    email = self.get_email('http://'+website)
                
                print([name, address, phone_number, coords[0], coords[1], website, email])
                
                
                row_index = len(self.worksheet.col_values(1)) + 1
                self.worksheet.update_acell('A'+str(row_index), name)
                self.worksheet.update_acell('B'+str(row_index), address)
                self.worksheet.update_acell('C'+str(row_index), phone_number) 
                self.worksheet.update_acell('D'+str(row_index), coords[0])
                self.worksheet.update_acell('E'+str(row_index), coords[1])
                self.worksheet.update_acell('F'+str(row_index), website)
                self.worksheet.update_acell('G'+str(row_index), email)
                
                element = self.driver.find_element_by_xpath("//button[contains(@class,'back-to-list')]")
                time.sleep(2)
                
                element.click()
                time.sleep(3)
            
        except Exception as e:
            print(e)
        
        time.sleep(10)
        #self.driver.quit()

        return(self.data)

query = "veterianrios murcia"
url = "https://www.google.com/maps?q="+query.replace(" ", "+")+"&hl=es"

gmaps = ScrapearGMaps()
print(gmaps.scrape(url))

     |████████████████████████████████| 911kB 4.3MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.5 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates In